## 调研员：从网络进行搜索并总结报告

### 背景
在MetaGPT中，调研员角色，可以根据用户的调研问题，从互联网上进行搜索总结，并最终生成报告。本文将从设计思路、代码实现、使用示例等几个方面介绍调研员角色。

### 目标
通过本文档，你可以了解如何使用MetaGPT的调研员角色进行网络搜索并总结报告，进一步地，可以使用MetaGPT的联网功能开发新的智能体

### 设计思路
在使用MetaGPT开发Researcher角色之前，我们需要先思考一下假如自己作为一个Researcher，在网络上搜索并输出调研报告是怎么做的。一般是包含以下几个步骤：

1. 分析待研究的问题，并将问题拆分成几个可以适合用搜索引擎进行搜索的子问题
2. 通过搜索引擎搜索子问题，浏览搜索引擎会给出多个带有标题、原文Url、摘要等信息的搜索结果，判断每一条搜索结果是否与要搜索的问题相关以及信息来源是否可靠，从而选择是否要进一步通过Url浏览网页
3. 点击需要进一步浏览的网页，判断网页内容对待研究的问题是否有帮助，提取有关的信息并记录
4. 聚合所有的记录下来的相关资料，针对待研究的问题撰写报告
因此，我们尝试让GPT模拟以上的调研流程，整体步骤如下：

1. 用户输入待调研问题
2. 调研员通过GPT生成一组研究问题，这些问题共同形成对任何给定任务的客观意见
3. 调研员在收到GPT分解的问题后，对于每个研究问题，先通过搜索引擎搜索，获取初次搜索结果
4. 网址通过浏览器获取网页内容，并使用对网页内容进行总结
5. 汇总所有总结后的内容并跟踪其来源
6. 最后，让GPT根据汇总后的内容生成最终研究报告

针对以上的流程，我们可以抽象出3个Action，定义一个Role，如下：


| 名称                         | 类型   | 说明                                         |
|--------------------------|------|------------------------------------------|
| CollectLinks                 | Action | 从搜索引擎进行搜索，并获取Url地址列表                  |
| WebBrowseAndSummarize | Action | 浏览网页并总结网页内容                           |
| ConductResearch           | Action | 生成调研报告                                   |
| Researcher                 | Role  | 调研员智能体，从网络进行搜索并总结报告                   |


所用的配置信息如下：
config/config2.yaml
```yaml
llm:
  api_type: 'ollama'
  base_url: 'http://192.168.0.123:11434/api'
  model: 'qwen2.5:7b'
  max_token: 4096
  
repair_llm_output: true

embedding:
  api_type: 'ollama'
  base_url: 'http://192.168.0.123:11434/api'
  model: 'qwen2.5:7b'

browser:
  engine: 'selenium'
  browser_type: 'chrome'
```

In [46]:
%%bash
mkdir -p config  # 创建 config 目录（若不存在）
cat <<EOF > config/config2.yaml
llm:
  api_type: 'zhipuai'
  api_key: '2ce2c9417f83dc0f582c00498b2bbd8b.374mpCJDIXF5ksmq'
  model: 'glm-4-flash'
  
repair_llm_output: true

embedding:
  api_type: 'zhipuai'
  api_key: '2ce2c9417f83dc0f582c00498b2bbd8b.374mpCJDIXF5ksmq'
  model: 'glm-4-flash'

browser:
  engine: 'selenium'
  browser_type: 'chrome'
EOF

由于metaGPT非常消耗token，所以我们用本地大模型qwen2.5:7b。然后搜索引擎我们稍后自己定制，不从配置导入，所以没有配置search。浏览器工具用selenium
所有的代码建议在jupyter notebook 中运行，这样可以随时看到一步步的输出结果，方便学习。


Action定义

CollectLinks

CollectLinks Action用于从搜索引擎搜索相关问题并获取Url地址列表。因为用户输入的问题不一定适合直接通过搜索引擎搜索，因此在进行搜索之前，先将用户输入的问题拆分成多个适合搜索的子问题，然后通过搜索引擎搜索这些子问题，筛选出与调研问题有关的Url，并根据网站可靠性对url列表进行排序。

metaGPT提供的几个搜索引擎都是国外的，必须要梯子，其实我们可以直接用智谱的，下面自定义一个智谱的搜索工具。

In [61]:
%%bash
cat <<EOF > .env
ZHIPU_API_KEY=<YOUR API KEY>
EOF

In [62]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key

from openai import OpenAI
from datetime import datetime
def zhipu_web_search_tool(query: str) -> str:
    """
    使用智谱AI的GLM-4模型进行联网搜索，返回搜索结果的字符串。
    
    参数:
    - query: 搜索关键词

    返回:
    - 搜索结果的字符串形式
    """
    # 初始化客户端
    client = OpenAI(
        api_key=os.getenv('ZHIPU_API_KEY'),
        base_url="https://open.bigmodel.cn/api/paas/v4/"
    ) 
    success = False

    # 获取当前日期
    current_date = datetime.now().strftime("%Y-%m-%d")

    print("current_date:", current_date)
    
    # 设置工具
    tools = [{
        "type": "web_search",
        "web_search": {
            "enable": True,
            "search_result": True
        }
    }]

    # 系统提示模板，包含时间信息
    system_prompt = f"""你是一个具备网络访问能力的智能助手，在适当情况下，优先使用网络信息（参考信息）来回答，
    以确保用户得到最新、准确的帮助。当前日期是 {current_date}。"""
        
    # 构建消息
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
        
    # 调用API
	# 因为智谱返回的结果有时候没有web_search，所以我们可以让它反复生成，直到生成web_search为止
    while success == False:
        response = client.chat.completions.create(
            model="glm-4-flash",
            messages=messages,
            tools=tools
        )
        try:
            rst = response.web_search
            success = True
        except:
            pass
        
    
    # 返回结果
    return rst

可以试一试这个搜索引擎接口好不好用？

In [64]:
rsp = zhipu_web_search_tool("最近三个月有哪些AI黑客松？")
print(rsp)

current_date: 2025-02-26
[{'content': '吴说获悉，Solana 宣布首届全球线上 AI 黑客松正式启动，将于 12 月 10 日至 23 日举行，奖金总额超 18.5 万美元，设立六大赛道聚焦 AI 代理开发及基础设施建设等创新领域。主赛道由 a16z crypto 和 Solana Foundation 支持，奖金最高 3 万美元，其余赛道涵盖 AI 基础设施、代币工具、自治聊天代理、Social/影响力者代理等，每个赛道由行业合作伙伴提供支持。参赛无需注册，作品提交将于 12 月 15 日开始。Solana 提供团队匹配、导师指导和资源支持，激励开发者构建基于 Solana 的 AI 创新应用。\n（转自：吴说）', 'icon': '', 'link': 'https://finance.sina.com.cn/blockchain/roll/2024-12-11/doc-incyzzcx0976575.shtml', 'media': '新浪财经', 'refer': 'ref_1', 'title': 'Solana 宣布首届全球线上 AI 黑客松正式启动'}, {'content': '本文来自：SendAI 负责人 Yash Agarwal\n编译Odaily星球日报\n译者Azuma\n编者按：由 SendAI 主办的 Solana AI 黑客松启动已有一定时日，但由于 AI 叙事的持续升温，社区对于相关项目的追捧热度丝毫不减。\n链上分析研究机构 Pine Analytics 昨日曾统计表示，由该场黑客松参与项目创建或直接关联的代币已多达 65 个，总市值为 4.82 亿美元，平均市值为 750 万美元，市值中位数为 170 万美元；有 13 个代币的市值超过 1000 万美元， 42 个代币的市值超过 100 万美元。\n然而，在社区情绪如此激昂的情况下，SendAI 创始人 Yash Agarwal 今日却发布了一篇长文抨击当前的炒作乱象 —— Yash 强调过去一周黑客松主页仅有 4 万次访问，但随随便便一个代币都能速通 5000 万市值SendAI 本想构建一个助推 Solana 生态 AI 发展的平台，但现在却成为了堕落者们的赌场”。在 Yash 来看，当前的泡沫会造成开发者趋向于盲目、短时，不利于 


然后我们可以把上面智谱这个搜索函数封装成metaGPT可以用的工具

In [75]:
from __future__ import annotations

import asyncio
import json
from concurrent import futures
from typing import Literal, Optional
from pydantic import BaseModel, ConfigDict
class ZhipuAPIWrapper(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    loop: Optional[asyncio.AbstractEventLoop] = None
    executor: Optional[futures.Executor] = None

    async def run(
        self,
        query: str,
        max_results: int = 5,
        as_string: bool = True,
    ) -> str | list[dict]:
        """Return the results of a Zhipu search using the official Google API

        Args:
            query: The search query.
            max_results: The number of results to return.
            as_string: A boolean flag to determine the return type of the results. If True, the function will
                return a formatted string with the search results. If False, it will return a list of dictionaries
                containing detailed information about each search result.

        Returns:
            The results of the search.
        """
        loop = self.loop or asyncio.get_event_loop()
        future = loop.run_in_executor(
            self.executor,
            self._search_from_zhipu,
            query,
            max_results,
        )
        search_results = await future
        if as_string:
            return json.dumps(search_results, ensure_ascii=False)
        return search_results

    def _search_from_zhipu(self, query: str, max_results: int):
        return [
            {"link": i["link"], "snippet": i["content"], "title": i["title"]}
            for (_, i) in zip(range(max_results), zhipu_web_search_tool(query))
        ]

上面这些代码是根据源代码 https://github.com/geekan/MetaGPT/blob/main/metagpt/tools/search_engine_ddg.py改写的，其实就是把里面的
self.ddgs.text(query) 换成了 zhipu_web_search_tool(query) 而已。

我们来试试这个封装起来的函数好不好用

In [66]:
run_func = ZhipuAPIWrapper().run
found = await run_func("上海最近三个月有哪些AI会议？", max_results=3, as_string=False)
print(len(found))

current_date: 2025-02-26
3


结果会显示 3
就是找到三条记录，其中每一条记录都是像下面这样的字典：
{
	'link': 'https://zhengce.qizhidao.com/article-zcview/707160600044830720.html', 
	'snippet': '知识产权是企业的核心竞争力之一，专利授权是其中最重要的组成部分之一。在拥有大量专利的情况下，企业能够更好地保护其研发成果和技术优势，从而提高市场份额和盈利能力。因此，专利授权率对申报知识产权优势企业的影响至关重要。\n专利授权率可以直接反映出企业的创新能力和技术实力。专利授权率越高，说明企业的创新成果越多，技术实力越强。这种技术实力是企业在竞争中不可或缺的优势，可以吸引更多的顶尖人才、合作伙伴和投资者，进一步促进企业的发展。\n专利授权率也可以影响到企业的商业模式和市场准入。在某些行业中，专利的重要性更加突出。例如，在药品制造业中，拥有专利的企业可以获得较长时间的独家生产权，从而保持市场优势。同样，在电子产品制造业中，拥有核心专利的企业可以控制市场标准，进一步增强市场地位。因此，专利授权率越高，企业在市场准入和商业模式方面的优势也就越大。\n专利授权率还可以影响到企业的资本运作。在融资过程中，拥有高专利授权率的企业通常更受投资者欢迎。这是因为专利授权率反映了企业的技术实力和创新能力，并且与企业的盈利能力直接相关。高专利授权率意味着企业拥有更多的收入来源和更强的竞争力，从而更容易获得资本市场的融资支持。\n综上所述，专利授权率对申报知识产权优势企业的影响非常重要。高专利授权率可以直接反映企业的创新能力和技术实力，进一步影响到企业的商业模式、市场准入和资本运作。因此，在知识产权战略规划中，提高专利授权率是企业应该考虑的重要因素之一，可以帮助企业获得更大的竞争优势。', 
	'title': '企业的专利授权率对申报知识产权优势企业有影响吗？-企知道'
}
每条数据都有link、snippet、title三个信息。

这个搜索工具现在是很好用的了，下面我们来构建Action。

In [82]:
from __future__ import annotations

import asyncio
from typing import Any, Callable, Optional, Union
from metagpt.logs import logger
from metagpt.config2 import config
from pydantic import TypeAdapter, model_validator
from metagpt.actions import Action
from metagpt.tools.search_engine import SearchEngine
from metagpt.utils.common import OutputParser
from metagpt.utils.text import generate_prompt_chunk, reduce_message_length

LANG_PROMPT = "请用{language}回答."

RESEARCH_BASE_SYSTEM = """你是一个专注于批判性思维的人工智能研究助理。你的唯一目标是根据给定的文本，撰写出结构严谨、客观公正、广受好评且文笔优美的报告。"""

RESEARCH_TOPIC_SYSTEM = "你是一位AI研究助理，你的研究课题是：\n#TOPIC#\n{topic}"

SEARCH_TOPIC_PROMPT:str = """你是一个研究助理，请你为你的研究课题{topic}提供最多2个必要的关键词用于网络搜索。你的回答必须是JSON格式，例如：["关键词1", "关键词2"]。"""

SUMMARIZE_SEARCH_PROMPT = """### 要求
与你的研究课题相关的关键词及搜索结果会在“搜索结果信息”部分显示。
### 搜索结果信息
{search_results}
根据搜索结果，提供最多 {decomposition_nums} 个与你的研究课题相关的查询。
请以以下JSON格式回应：["query1", "query2", "query3", ...]。除了python的list形式的JSON格式不要输出任何其他内容。
"""

COLLECT_AND_RANKURLS_PROMPT = """### 主题
{topic}
### 查询
{query}
### 在线搜索结果
{results}
### 要求
请移除与查询或主题无关的搜索结果。
然后，根据链接的可信度对剩余的搜索结果进行排序。
如果两个结果的可信度相同，则根据相关性优先级排序。
提供排序后结果的索引，采用JSON格式，例如 [0, 1, 3, 4, ...]，不包含其他文字。
"""

class CollectLinks(Action):
    """Action class to collect links from a search engine."""

    name: str = "CollectLinks"
    i_context: Optional[str] = None
    desc: str = "Collect links from a search engine."
    search_func: Optional[Any] = None
    search_engine: Optional[SearchEngine] = None
    rank_func: Optional[Callable[[list[str]], None]] = None

    @model_validator(mode="after")
    def validate_engine_and_run_func(self):
        if self.search_engine is None:
            self.search_engine = SearchEngine.from_search_func(ZhipuAPIWrapper().run)
        return self

    async def run(
        self,
        topic: str,
        decomposition_nums: int = 4,
        url_per_query: int = 4,
        system_text: str | None = None,
    ) -> dict[str, list[str]]:
        """Run the action to collect links.

        Args:
            topic: The research topic.
            decomposition_nums: The number of search questions to generate.
            url_per_query: The number of URLs to collect per search question.
            system_text: The system text.

        Returns:
            A dictionary containing the search questions as keys and the collected URLs as values.
        """
        system_text = system_text if system_text else RESEARCH_TOPIC_SYSTEM.format(topic=topic)
        keywords = await self._aask(SEARCH_TOPIC_PROMPT.format(topic=topic))
        try:
            keywords = OutputParser.extract_struct(keywords, list)
            keywords = TypeAdapter(list[str]).validate_python(keywords)
        except Exception as e:
            logger.exception(f"fail to get keywords related to the research topic '{topic}' for {e}")
            keywords = [topic]
        results = await asyncio.gather(*(self.search_engine.run(i, as_string=False) for i in keywords))

        def gen_msg():
            while True:
                search_results = "\n".join(
                    f"#### Keyword: {i}\n Search Result: {j}\n" for (i, j) in zip(keywords, results)
                )
                prompt = SUMMARIZE_SEARCH_PROMPT.format(
                    decomposition_nums=decomposition_nums, search_results=search_results
                )
                yield prompt
                remove = max(results, key=len)
                remove.pop()
                if len(remove) == 0:
                    break

        model_name = 'gpt-4'
        prompt = reduce_message_length(gen_msg(), model_name, system_text, config.llm.max_token)
        logger.debug(prompt)
        queries = await self._aask(prompt, [system_text])
        try:
            queries = OutputParser.extract_struct(queries, list)
            queries = TypeAdapter(list[str]).validate_python(queries)
        except Exception as e:
            logger.exception(f"fail to break down the research question due to {e}")
            queries = keywords
        ret = {}
        for query in queries:
            ret[query] = await self._search_and_rank_urls(topic, query, url_per_query)
        return ret

    async def _search_and_rank_urls(self, topic: str, query: str, num_results: int = 2) -> list[str]:
        """Search and rank URLs based on a query.

        Args:
            topic: The research topic.
            query: The search query.
            num_results: The number of URLs to collect.

        Returns:
            A list of ranked URLs.
        """
        max_results = max(num_results * 2, 5)
        results = await self.search_engine.run(query, max_results=max_results, as_string=False)
        if len(results) == 0:
            return []
        _results = "\n".join(f"{i}: {j}" for i, j in zip(range(max_results), results))
        prompt = COLLECT_AND_RANKURLS_PROMPT.format(topic=topic, query=query, results=_results)
        logger.debug(prompt)
        indices = await self._aask(prompt)
        try:
            indices = OutputParser.extract_struct(indices, list)
            assert all(isinstance(i, int) for i in indices)
        except Exception as e:
            logger.exception(f"fail to rank results for {e}")
            indices = list(range(max_results))
        results = [results[i] for i in indices]
        if self.rank_func:
            results = self.rank_func(results)
        return [i["link"] for i in results[:num_results]]

需要注意的几个地方是：

我们通过 self.search_engine = SearchEngine.from_search_func(ZhipuAPIWrapper().run) 这行代码，把自定义的搜索工具传给Action。
在运行reduce_message_length之前，我们需要改一下model_name
model_name = 'gpt-4'

因为这个函数会利用

In [85]:
# import tiktoken
# model_name = "qwen2.5:7b"
# # 假设 "qwen2.5:7b" 使用与 "gpt-4" 相同的编码器
# # encoding_name = "gpt-4"  # 或者其他已知与 "qwen2.5:7b" 兼容的编码器名称
# encoding = tiktoken.get_encoding(encoding_name)

来计算某个字符串对应多少token。而tiktoken只认得OpenAI家的模型，它不认得我们本地的qwen2.5:7b，会报错，所以我们改一下模型名称，不让他报错。


我们现在来试试这个CollectLinks好用不？

In [84]:
collect_links = CollectLinks()
links = await collect_links.run("专利实施例数量对于授权率的影响")
print(links)

```json
["专利实施例数量", "授权率影响

2025-02-26 17:08:46.933 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.


"]
```
current_date: 2025-02-26
current_date: 2025-02-26
["专利实施例数量如何影响专利授权率", "专利说明书中的实施例对授权率有何影响", "专利实施例数量与专利授权质量的关系", "专利实施例的详细

2025-02-26 17:09:08.996 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.


程度对授权率的影响"]
current_date: 2025-02-26
```json
[0, 1, 3, 2, 4, 5, 6]
```

...


current_date: 2025-02-26
```json
[2, 0, 1, 3

2025-02-26 17:10:20.908 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.


, 5]
```
current_date: 2025-02-26
```json
[0, 1,

2025-02-26 17:10:39.796 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.


 3, 7]
```
{'专利实施例数量如何影响专利授权率': ['https://zhuanlan.zhihu.com/p/635969428', 'https://www.110ask.com/kuaiwen/10009713880039775605.html', 'https://zhuanlan.zhihu.com/p/657425833', 'https://www.66law.cn/laws/2875459.aspx'], '专利说明书中的实施例对授权率有何影响': ['https://zhuanlan.zhihu.com/p/635969428', 'https://wenwen.sogou.com/z/q808529523.htm', 'https://wenwen.sogou.com/z/q728455154.htm', 'https://zhuanlan.zhihu.com/p/661628898'], '专利实施例数量与专利授权质量的关系': ['https://zhuanlan.zhihu.com/p/660518632', 'https://www.lunwen881.com/zl/246397.html', 'https://m.makepolo.com/product/100431184329.html', 'https://www.lunwen881.com/zl/209952.html'], '专利实施例的详细程度对授权率的影响': ['https://zhuanlan.zhihu.com/p/635969428', 'https://zhuanlan.zhihu.com/p/660518632', 'https://www.66law.cn/laws/2875459.aspx', 'https://www.zhihu.com/question/589219777']}


输出了
{'申请专利哪些因素影响授权率': ['http://mp.weixin.qq.com/s?__biz=MzAxODAzNjIxMQ==&mid=2650189385&idx=2&sn=3b80b4796f330cc818ed626fb9c3046a',
  'https://zhuanlan.zhihu.com/p/660518632'],
 '哪些因素影响专利申请授权率？': ['http://mp.weixin.qq.com/s?__biz=MzkyMTI0OTA3MQ==&mid=2247512783&idx=3&sn=78ab46ff135f29fef20f3de78ec154bd',
  'https://zhuanlan.zhihu.com/p/625021220'],
 '企业的专利授权率对申报知识产权优势企业有影响吗？': ['http://wap.qikanzj.com/p-37224',
  'https://zhengce.qizhidao.com/article-zcview/707160600044830720.html']}
  
说明这个Action没问题。我们继续配置下一个Action

In [86]:
from metagpt.tools.web_browser_engine import WebBrowserEngine
WEB_BROWSE_AND_SUMMARIZE_PROMPT = """
### 要求
利用“参考信息”部分的文本回答问题“{query}”。
如果问题无法直接用文本回答，但文本与研究课题相关，请提供一个全面的文本摘要。
如果文本完全与研究课题无关，请回复简短文字“不相关”。
包含所有相关的事实信息、数字、统计数据等（如果有的话）。
### 参考信息
{content}
"""
RESEARCH_TOPIC_SYSTEM = "你是一位AI研究助理，你的研究课题是：\n#TOPIC#\n{topic}"

RESEARCH_BASE_SYSTEM = """你是一个专注于批判性思维的人工智能研究助理。你的唯一目标是根据给定的文本，撰写出结构严谨、客观公正、广受好评且文笔优美的报告。"""

class WebBrowseAndSummarize(Action):
    """Action class to explore the web and provide summaries of articles and webpages."""

    name: str = "WebBrowseAndSummarize"
    i_context: Optional[str] = None
    desc: str = "Explore the web and provide summaries of articles and webpages."
    browse_func: Union[Callable[[list[str]], None], None] = None
    web_browser_engine: Optional[WebBrowserEngine] = None

    @model_validator(mode="after")
    def validate_engine_and_run_func(self):
        if self.web_browser_engine is None:
            self.web_browser_engine = WebBrowserEngine.from_browser_config(
                self.config.browser,
                browse_func=self.browse_func
            )
        return self

    async def run(
        self,
        url: str,
        *urls: str,
        query: str,
        system_text: str = RESEARCH_BASE_SYSTEM,
    ) -> dict[str, str]:
        """Run the action to browse the web and provide summaries.

        Args:
            url: The main URL to browse.
            urls: Additional URLs to browse.
            query: The research question.
            system_text: The system text.

        Returns:
            A dictionary containing the URLs as keys and their summaries as values.
        """
        contents = await self.web_browser_engine.run(url, *urls)
        if not urls:
            contents = [contents]

        summaries = {}
        prompt_template = WEB_BROWSE_AND_SUMMARIZE_PROMPT.format(query=query, content="{}")
        for u, content in zip([url, *urls], contents):
            content = content.inner_text
            chunk_summaries = []
            for prompt in generate_prompt_chunk(content, prompt_template, "gpt-4", system_text, 4096):
                logger.debug(prompt)
                summary = await self._aask(prompt, [system_text])
                if summary in "不相关。":
                    continue
                chunk_summaries.append(summary)

            if not chunk_summaries:
                summaries[u] = None
                continue

            if len(chunk_summaries) == 1:
                summaries[u] = chunk_summaries[0]
                continue

            content = "\n".join(chunk_summaries)
            prompt = WEB_BROWSE_AND_SUMMARIZE_PROMPT.format(query=query, content=content)
            summary = await self._aask(prompt, [system_text])
            summaries[u] = summary
        return summaries

上面代码也是从https://github.com/geekan/MetaGPT/blob/main/metagpt/actions/research.py 源码中拷贝的，改了几个地方：
proxy=self.config.proxy, 删掉了，因为我们不用梯子，不需要代理。
if summary == "Not relevant." 改成了 if summary in "不相关。"

还有提示词都改成中文了，其他代码不变。

然后我们来试一下这个Action好用不？

In [ ]:
vbs = WebBrowseAndSummarize()
url = 'http://mp.weixin.qq.com/s?__biz=MzIyOTg5MTA3MQ==&mid=2247503955&idx=1&sn=fb1056b09a06db4b0d3264ad811f640c'
vbs_rst = await vbs.run(url,query="专利实施例数量对于授权率的影响")
print(vbs_rst)

可见这是一个字典，链接是键，内容是值。

然后我们继续来构建ConductResearch这个Action。

In [ ]:
CONDUCT_RESEARCH_PROMPT = """### 参考信息
{content}
### 要求
请根据上述提供的信息，针对以下主题：“{topic}”，撰写一份详细的研究报告。该报告必须满足以下要求：
直接围绕选定的主题展开讨论。
确保结构严谨、内容深入，尽可能纳入相关事实和数据。
以直观的方式呈现数据和研究结果，如有适用，可以使用特性比较表。
报告的字数不少于2,000字，并采用Markdown语法进行格式化，遵循APA风格指南。
在报告末尾以APA格式列出所有来源的URL。
"""

class ConductResearch(Action):
    """Action class to conduct research and generate a research report."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    async def run(
        self,
        topic: str,
        content: str,
        system_text: str = RESEARCH_BASE_SYSTEM,
    ) -> str:
        """Run the action to conduct research and generate a research report.

        Args:
            topic: The research topic.
            content: The content for research.
            system_text: The system text.

        Returns:
            The generated research report.
        """
        prompt = CONDUCT_RESEARCH_PROMPT.format(topic=topic, content=content)
        logger.debug(prompt)
        self.llm.auto_max_tokens = True
        return await self._aask(prompt, [system_text])


def get_research_system_text(topic: str, language: str):
    """Get the system text for conducting research.

    Args:
        topic: The research topic.
        language: The language for the system text.

    Returns:
        The system text for conducting research.
    """
    return " ".join((RESEARCH_TOPIC_SYSTEM.format(topic=topic), LANG_PROMPT.format(language=language)))

这个就是把prompt改成中文的了，其他都是源码的内容。
我们再来试试这个Action好用不？

In [ ]:
conduct_research = ConductResearch()
report = await conduct_research.run(topic="专利实施例数量对于授权率的影响", content=str(vbs_rst))
print(report)

输出了：
```markdown
# 专利实施例数量对于授权率的影响研究报告

## 引言

专利是创新成果的重要体现形式之一。随着知识产权保护意识的增强，越来越多的企业和个人开始重视专利申请。然而，在实际操作过程中，许多申请人关心的一个重要问题是：专利实施例的数量是否会影响其最终的授权率？本文旨在探讨这一问题，并通过分析相关文献和数据来揭示专利实施例数量与授权率之间的关系。

## 文献综述

### 专利授权的基本概念

根据《中华人民共和国专利法》的规定，发明创造必须具备新颖性、创造性及实用性才能获得专利权。在实际审查过程中，审查员会综合考虑申请文件中描述的技术方案是否满足上述条件来决定是否授予专利权。

### 实施例的作用与重要性

实施例是专利说明书中的一个重要组成部分，它通过具体实例详细说明了发明创造的技术方案及其应用方式。实施例不仅有助于提高说明书的可读性和理解度，还能为审查员提供更加直观、具体的参考依据，从而帮助其更好地评估申请文件所描述的技术方案是否满足授权条件。

## 研究方法

### 数据收集

为了进行深入研究，我们从多个公开渠道收集了大量专利申请案例数据。这些数据涵盖了不同技术领域和不同规模的企业提交的专利申请情况，并记录了每项申请中包含的具体实施例数量及其最终的授权结果。

### 分析工具与步骤

1. **数据清洗**：对原始数据进行预处理，包括去除重复值、修正错误信息等。
2. **统计分析**：使用描述性统计方法计算各类专利申请案例的基本特征（如平均实施例数）；通过相关性和回归分析探索实施例数量与授权率之间的关系。
3. **可视化展示**：利用图表等形式直观地呈现研究结果。

## 研究结果

### 实施例数量与授权率的相关性

通过对收集到的数据进行统计分析，我们发现实施例数量与专利授权率之间存在显著相关性。具体而言，在其他条件相同的情况下：

- 申请文件中包含更多实施例的专利申请更有可能获得授权。
- 每增加一个实施例，专利授权的可能性大约提高5%至10%。

### 实施例质量的重要性

值得注意的是，仅仅增加实施例的数量并不足以保证更高的授权率。高质量、详尽且具有代表性的实施例对于提升授权成功率同样至关重要。例如，在某些情况下，虽然增加了多个实施例，但由于这些新增加的实例缺乏新颖性或与主要技术方案关联度不高，反而可能降低整体申请文件的质量。

## 讨论

### 专利审查过程中的考量因素

在实际审查过程中，审查员会综合考虑多项因素来决定是否授予专利权。除了实施例的数量外，还包括但不限于以下几点：

- 技术新颖性和创新性
- 实用性与技术方案的可行性
- 权利要求书的撰写质量
- 说明书的整体清晰度和完整性

### 对企业策略的影响

上述研究结果对企业制定专利申请策略具有重要指导意义。一方面，增加实施例数量可以提高授权率；但另一方面，必须确保这些新增加的内容具有实际价值并能够有效支持主要技术方案。

## 结论与建议

综上所述，专利实施例的数量确实会对最终的授权率产生影响。然而，在追求更多实施例的同时，企业还应注重提升其撰写质量和代表性，以确保整个申请文件的质量和完整性。此外，对于代理机构而言，则需更加关注如何平衡提高授权率的需求与保持技术方案创新性的目标。

## 参考文献

- [http://mp.weixin.qq.com/s?__biz=MzIyOTg5MTA3MQ==&mid=2247503955&idx=1&sn=fb1056b09a06db4b0d3264ad811f640c](http://mp.weixin.qq.com/s?__biz=MzIyOTg5MTA3MQ==&mid=2247503955&idx=1&sn=fb1056b09a06db4b0d3264ad811f640c)

---

请注意，上述报告是基于提供的参考信息进行的模拟撰写，并未引用实际的研究数据或文献。在正式研究中，应通过查阅相关学术论文、官方统计数据等资源来获取更准确的信息
2025-02-12 15:05:02.306 | INFO     | metagpt.utils.cost_manager:update_cost:108 - prompt_tokens: 432, completion_tokens: 948
和结论。
```
最后我们来构建Researcher这个Role。



In [ ]:
import asyncio
import re

from pydantic import BaseModel

from metagpt.const import RESEARCH_PATH
from metagpt.logs import logger
from metagpt.roles.role import Role, RoleReactMode
from metagpt.schema import Message


class Report(BaseModel):
    topic: str
    links: dict[str, list[str]] = None
    summaries: list[tuple[str, str]] = None
    content: str = ""


class Researcher(Role):
    name: str = "大卫"
    profile: str = "研究员"
    goal: str = "收集信息并进行研究"
    constraints: str = "保证信息的真实和准确"
    language: str = "zh-cn"
    enable_concurrency: bool = True

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([CollectLinks, WebBrowseAndSummarize, ConductResearch])
        self._set_react_mode(RoleReactMode.BY_ORDER.value, len(self.actions))
        if self.language not in ("en-us", "zh-cn"):
            logger.warning(f"The language `{self.language}` has not been tested, it may not work.")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo
        msg = self.rc.memory.get(k=1)[0]
        if isinstance(msg.instruct_content, Report):
            instruct_content = msg.instruct_content
            topic = instruct_content.topic
        else:
            topic = msg.content

        research_system_text = self.research_system_text(topic, todo)
        if isinstance(todo, CollectLinks):
            links = await todo.run(topic, 4, 4)
            ret = Message(
                content="", instruct_content=Report(topic=topic, links=links), role=self.profile, cause_by=todo
            )
        elif isinstance(todo, WebBrowseAndSummarize):
            links = instruct_content.links
            todos = (
                todo.run(*url, query=query, system_text=research_system_text) for (query, url) in links.items() if url
            )
            if self.enable_concurrency:
                summaries = await asyncio.gather(*todos)
            else:
                summaries = [await i for i in todos]
            summaries = list((url, summary) for i in summaries for (url, summary) in i.items() if summary)
            ret = Message(
                content="", instruct_content=Report(topic=topic, summaries=summaries), role=self.profile, cause_by=todo
            )
        else:
            summaries = instruct_content.summaries
            summary_text = "\n---\n".join(f"url: {url}\nsummary: {summary}" for (url, summary) in summaries)
            content = await self.rc.todo.run(topic, summary_text, system_text=research_system_text)
            ret = Message(
                content="",
                instruct_content=Report(topic=topic, content=content),
                role=self.profile,
                cause_by=self.rc.todo,
            )
        self.rc.memory.add(ret)
        return ret

    def research_system_text(self, topic, current_task: Action) -> str:
        """BACKWARD compatible
        This allows sub-class able to define its own system prompt based on topic.
        return the previous implementation to have backward compatible
        Args:
            topic:
            language:

        Returns: str
        """
        return get_research_system_text(topic, self.language)

    async def react(self) -> Message:
        msg = await super().react()
        report = msg.instruct_content
        self.write_report(report.topic, report.content)
        return msg

    def write_report(self, topic: str, content: str):
        filename = re.sub(r'[\\/:"*?<>|]+', " ", topic)
        filename = filename.replace("\n", "")
        if not RESEARCH_PATH.exists():
            RESEARCH_PATH.mkdir(parents=True)
        filepath = RESEARCH_PATH / f"{filename}.md"
        filepath.write_text(content)

这也是直接拷贝的源码。把英文改成中文而已，其他没变。

然后我们再定义一个main函数

In [ ]:
async def main(topic: str, language: str = "zh-cn", enable_concurrency: bool = True):
    role = Researcher(language=language, enable_concurrency=enable_concurrency)
    await role.run(topic)

最后去在jupyter notebook中运行这个main函数，就会开始写报告了

In [ ]:
await main("中小企业大模型应用落地的最佳实践")

到此为止我们就完成了metaGPT的调研员这个多动作单智能体的学习。这里需要注意的是：

1. 搜索工具换成了智谱的
2. prompt换成中文的
3. Action里的model_name换成"gpt-4"避免报错。
4. 每构建完一个工具、动作、角色，都测试一下它好用不？